In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install typing

     |████████████████████████████████| 78 kB 3.5 MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=4337d1d6c38f1967e95e5fc9160a2d1abd32c352bafe381387cec3d738194d17
  Stored in directory: /root/.cache/pip/wheels/35/f3/15/01aa6571f0a72ee6ae7b827c1491c37a1f72d686fd22b43b0e
Successfully built typing


In [3]:
# LIBRARIES 
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from scipy import stats

In [5]:
data = pd.read_csv("/content/drive/MyDrive/MiniProject/Dataset/papers.csv")
# data = pd.read_csv("/content/drive/MyDrive/papers.csv")
data.head() # DISPLAYS FIRST 5 DATA COLOUMNS
data.columns.str.match("Unnamed")
data.loc[:,~data.columns.str.match("Unnamed")]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,title,abstract,id,author,journal,doi,update
0,Calculation of prompt diphoton production cros...,A fully differential calculation in perturbat...,704,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,2008-11-26
1,Polymer Quantum Mechanics and its Continuum Limit,A rather non-standard quantum representation ...,704.001,"Alejandro Corichi, Tatjana Vukasinac and Jose ...","Phys.Rev.D76:044016,2007",10.1103/PhysRevD.76.044016,2008-11-26
2,Numerical solution of shock and ramp compressi...,A general formulation was developed to repres...,704.001,Damian C. Swift,"Journal of Applied Physics, vol 104, 073536 (2...",10.1063/1.2975338,2009-02-05
3,"The Spitzer c2d Survey of Large, Nearby, Inste...",We discuss the results from the combined IRAC...,704.001,"Paul Harvey, Bruno Merin, Tracy L. Huard, Luis...","Astrophys.J.663:1149-1173,2007",10.1086/518646,2010-03-18
4,Fermionic superstring loop amplitudes in the p...,The pure spinor formulation of the ten-dimens...,704.001,Christian Stahn,"JHEP 0705:034,2007",10.1088/1126-6708/2007/05/034,2009-11-13
...,...,...,...,...,...,...,...
651569,Towards a Microscopic Theory for Metallic Heav...,The bias-dependent resistance R(V) of NS-junc...,supr-con/9607007,F. B. Anders and K. Gloos,"Physica B, 230-232, 437 (1997)",10.1016/S0921-4526(96)00604-7,2009-10-30
651570,Extended bound states and resonances of two fe...,The high-$T_c$ cuprates are possible candidat...,supr-con/9608003,"A. S. Blaer (1), H. C. Ren (2), and O. Tcherny...","Phys. Rev. B 55, 6035 (1997)",10.1103/PhysRevB.55.6035,2009-10-30
651571,Ginzburg-Landau-Gor'kov Theory of Magnetic osc...,We investigate de Haas-van Alphen (dHvA) osci...,supr-con/9608004,"G. M. Bruun, V. N. Nicopoulos, N. F. Johnson",PHYS REV B 56: (2) 809-825 JUL 1 1997,10.1103/PhysRevB.56.809,2016-09-08
651572,Evidence for Quasiparticle Decay in Photoemiss...,I argue that the ``gap'' recently observed at...,supr-con/9608005,R. B. Laughlin (Stanford),"Phys. Rev. Lett. 79, 1726 (1997)",10.1103/PhysRevLett.79.1726,2009-10-30


Semantic Scholar API to Obtain Metadata

In [6]:
from typing_extensions import Literal
import requests


class SemanticScholar:

    DEFAULT_API_URL = 'https://api.semanticscholar.org/v1'
    DEFAULT_PARTNER_API_URL = 'https://partner.semanticscholar.org/v1'

    auth_header = {}

    def __init__(self,timeout: int=2,api_key: str=None,api_url: str=None) -> None:

        if api_url:
            self.api_url = api_url
        else:
            self.api_url = self.DEFAULT_API_URL

        if api_key:
            self.auth_header = {'x-api-key': api_key}
            if not api_url:
                self.api_url = self.DEFAULT_PARTNER_API_URL

        self.timeout = timeout

    def paper(self, id: str, include_unknown_refs: bool=False) -> dict:
        # Paper lookup
        # param str id: S2PaperId, DOI or ArXivId.
        data = self.get_data('paper', id, include_unknown_refs)
        return data

    def author(self, id: str) -> dict:
        data = self.get_data('author', id, False)
        return data

    def get_data(self,method: Literal['paper', 'author'],id: str ,include_unknown_refs: bool) -> dict:

        data = {}
        method_types = ['paper', 'author']
        if method not in method_types:
            raise ValueError(
                'Invalid method type. Expected one of: {}'.format(method_types)
            )

        url = '{}/{}/{}'.format(self.api_url, method, id)
        if include_unknown_refs:
            url += '?include_unknown_references=true'
        r = requests.get(url, timeout=self.timeout, headers=self.auth_header)

        if r.status_code == 200:
            data = r.json()
            if len(data) == 1 and 'error' in data:
                data = {}
        elif r.status_code == 403:
            raise PermissionError('HTTP status 403 Forbidden.')
        elif r.status_code == 429:
            raise ConnectionRefusedError('HTTP status 429 Too Many Requests.')

        return data

Obtain MetaData Information About the Paper

In [32]:
sch = SemanticScholar(timeout=2)
# paper = sch.paper('10.1093/mind/lix.236.433')
# paper = sch.paper('10.1063/1.2975338')
# paper.keys()
# print(paper['doi'])
# for author in paper['authors']:
#   print(author['name'])
#   print(author['authorId'])

# for st in paper.keys():
#  print(st,':',paper[st]) 

# for i in range(0,len(data)): # RUN THIS AND TAB BELOW LINES TO RUN FOR ENTIRE DATASET - NOT REQUIRED
paper = sch.paper(data.doi[1])
for st in paper.keys():
  print(st,':',paper[st]) 


abstract : A rather nonstandard quantum representation of the canonical commutation relations of quantum mechanics systems, known as the polymer representation, has gained some attention in recent years, due to its possible relation with Planck scale physics. In particular, this approach has been followed in a symmetric sector of loop quantum gravity known as loop quantum cosmology. Here we explore different aspects of the relation between the ordinary Schroedinger theory and the polymer description. The paper has two parts. In the first one, we derive the polymer quantum mechanics starting from the ordinary Schroedinger theory and show that the polymer description arises as an appropriate limit. In the second part we consider the continuum limit of this theory, namely, the reverse process in which one starts from the discrete theory and tries to recover back the ordinary Schroedinger quantum mechanics. We consider several examples of interest, including the harmonic oscillator, the fr

ANALYSIS OF METADATA OF SINGLE PAPER 

In [39]:
def data_print(paper) -> None :
  print("Basic Metadata")
  print("Title:" , str(paper['title']))
  print("Fields of Study:" , str(paper['fieldsOfStudy']))
  print("Number Cited by:" , str(paper['numCitedBy']))
  print("Number Citing:" ,str(paper['numCiting']))

data_print(paper)  

Basic Metadata
Title: Polymer Quantum Mechanics and its Continuum Limit
Fields of Study: ['Physics']
Number Cited by: 124
Number Citing: 20


In [8]:
# API WITH KEYWORDS IN IT
import requests
resp = requests.get('http://api.springernature.com/metadata/json/doi/10.1007/s11276-008-0131-4?api_key=78efc06559822ae52f1e57a4fde9aab6')
print(resp.text)

{"apiMessage":"This JSON was provided by Springer Nature","query":"doi:10.1007/s11276-008-0131-4","apiKey":"78efc06559822ae52f1e57a4fde9aab6","result":[{"total":"1","start":"1","pageLength":"10","recordsDisplayed":"1"}],"records":[{"contentType":"Article","identifier":"doi:10.1007/s11276-008-0131-4","language":"en","url":[{"format":"","platform":"","value":"http://dx.doi.org/10.1007/s11276-008-0131-4"}],"title":"Bit-per-joule performance of power saving ad hoc networks with a mobile backbone","creators":[{ "creator":"Huang, Xiaolong" },{ "creator":"Rubin, Izhak" }],"publicationName":"Wireless Networks","openaccess":"false","doi":"10.1007/s11276-008-0131-4","publisher":"Springer","publicationDate":"2010-02-01","publicationType":"Journal","issn":"1572-8196","volume":"16","number":"2","genre":"OriginalPaper","startingPage":"311","endingPage":"329","journalId":"11276","copyright":"©2008 Springer Science+Business Media, LLC","abstract":"Energy efficient MAC protocols have been developed for